In [1]:
from tika import parser
import pandas as pd
from io import StringIO
import re

Desde el 32 al 34 > Únicamente texto

In [3]:
raw = parser.from_file('./pdf_data/Actualizacion_32_COVID-19.pdf')

In [3]:
ind_ini = raw['content'].find('en España:')
ind_fin = raw['content'].find('mundo:')

In [4]:
print(raw['content'][ind_ini:ind_fin])

en España: 
 
En esta semana se han detectado un total de 14 casos en España. Once de ellos tienen relación 
epidemiológica con el brote de Italia, y 3 permanecen en investigación.  
 
Hasta el momento se han registrado 16 casos: 6 en las Islas Canarias, 2 en la Comunidad Valenciana, 2 en 
Cataluña, 4 en Madrid, 1 en Andalucía y 1 en Baleares.  
 
Situación en el 


35 diferente

In [5]:
raw = parser.from_file('./pdf_data/Actualizacion_37_COVID-19.pdf')

fecha

In [2]:
def get_fecha(string):
    ind_ini = string.find('(COVID-19)')
    ind_fin = string.find('2020')
    return string[ind_ini:ind_fin].split('\n')[-1]+'2020'

In [7]:
fecha = get_fecha(raw['content'])

In [3]:
def parsing_table(string):
    first_number = re.search(r'\d+', string).group()
    new_lst = string.split(' ')
    ind = new_lst.index(first_number)
    final_list = [''.join(new_lst[:ind])]+new_lst[ind:]
    return [el for el in final_list if el != '']

In [4]:
def get_lst(string):
    ind_ini = string.lower().find('andalucía')
    ind_fin = string[ind_ini:].lower().find('total')
    list_tab = string[ind_ini:ind_ini+ind_fin].split('\n')
    list_tab = [el.replace(',','.') for el in [el.rstrip() for el in list_tab] if el != '']
    return [parsing_table(el) for el in list_tab]

In [5]:
def save_csv(lst, path):
    lst = get_lst(raw['content'])
    if len(lst[0])==5:
        cols = ['CCAA', 'casos', 'IA', 'UCI', 'muertes']
    elif len(lst[0])==4:
        cols = ['CCAA', 'casos', 'IA', 'UCI']
    elif len(lst[0])==3:
        cols = ['CCAA', 'casos', 'IA']
    data = pd.DataFrame(lst, columns = cols)
    data['fecha'] = [fecha]*data.shape[0]
    cols.insert(1, 'fecha')
    data[cols].to_csv(path, index=False)

In [7]:
for i in range(42,43):
    print(i)
    raw = parser.from_file('./pdf_data/Actualizacion_{}_COVID-19.pdf'.format(i))
    fecha = get_fecha(raw['content'])
    lst = get_lst(raw['content'])
    save_csv(lst,'./csv_data/COVID_es_{}.csv'.format(fecha.replace('.','_')))

42
